In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ="/home/tdelatte/new-projects/ethereum-analytics/key/ethereum-analytics-309308-6c01508bc0b8.json"

In [3]:
from google.cloud import bigquery
client = bigquery.Client()

In [4]:
import pandas as pd

In [5]:
def load_data_from_bigquery(QUERY):
    
    query_job = client.query(QUERY) # API request
    df = query_job.to_dataframe()
    
    return df

In [6]:
QUERY = """

        WITH
          ethereum_balance AS (
          SELECT
            address AS ethereum_address,
            (eth_balance / POWER(10, 18)) AS eth_balance
          FROM
            `bigquery-public-data.crypto_ethereum.balances`
          WHERE
            (eth_balance / POWER(10, 18)) > 1
          LIMIT
            10000),
          
          top_tokens AS (
          SELECT
            token_address,
            COUNT(1) AS transfer_count
          FROM
            `bigquery-public-data.ethereum_blockchain.token_transfers` AS token_transfers
          GROUP BY
            token_address
          ORDER BY
            transfer_count DESC
          LIMIT
            10000),
         
         token_balances AS (
          WITH
            double_entry_book AS (
            SELECT
              token_address,
              to_address AS ethereum_address,
              CAST(value AS float64) AS value,
              block_timestamp,
              transaction_hash
            FROM
              `bigquery-public-data.ethereum_blockchain.token_transfers`
            UNION ALL
            SELECT
              token_address,
              from_address AS ethereum_address,
              -CAST(value AS float64) AS value,
              block_timestamp,
              transaction_hash
            FROM
              `bigquery-public-data.ethereum_blockchain.token_transfers` )
          SELECT
            a.ethereum_address,
            b.token_address,
            SUM(value) AS balance,
            COUNT(DISTINCT transaction_hash) as unique_transfers
          FROM
            ethereum_balance a
          JOIN
            double_entry_book b
          ON
            a.ethereum_address = b.ethereum_address
          JOIN
            top_tokens c
          ON
            c.token_address = b.token_address
          WHERE
            a.ethereum_address != '0x0000000000000000000000000000000000000000'
          GROUP BY
            1,
            2
          HAVING
            balance > 0 )
            
            
        SELECT
          ethereum_address,
          MAX(eth_balance) AS ether_balance,
          COUNT(DISTINCT token_address) AS unique_tokens,
          MAX(unique_transfers) AS unique_transfers
        FROM
          ethereum_balance a
        JOIN
          token_balances b
        USING
          (ethereum_address)
        GROUP BY
          1

    """

In [ ]:
eth_dataset = load_data_from_bigquery(QUERY)

In [ ]:
eth_dataset.head(10)

In [ ]:
from etherscan import Etherscan
api_key = os.environ.get("ETHERSCAN_API_KEY")
eth = Etherscan(api_key) # key in quotation marks

In [ ]:
eth = Etherscan("UJZSE9DXM1T37JZQVNQYNSS9IT5JEPIBA7") # key in quotation marks

In [ ]:
eth_dataset["mined_blocks"] = 0

In [ ]:
eth_dataset.head()

In [ ]:
dataset_sample = eth_dataset[0:10]

In [ ]:
for i, row in dataset_samplet.iterrows():
    dataset_sample = row.ethereum_address
    try:
        mined = len(eth.get_mined_blocks_by_address(address=eth_address))
    except:
        continue
    if mined:
        row["mined_blocks"] += mined

In [ ]:
labeled_dataset = pd.read_csv("../data/external/eth_addresses.csv")

In [ ]:
labeled_dataset = labeled_dataset[labeled_dataset.Entity != "Entity"]

In [ ]:
labels.rename({'Address': 'ethereum_address'}, axis=1, inplace=True)

In [ ]:
ethdataset["Label"] = "Unknown"

In [ ]:
labeled_dataset = pd.merge(eth_dataset, labels)

In [ ]:
labeled_dataset.Entity.value_counts()

In [ ]:
labeled_dataset.head(20)

In [ ]:
# Manually add crowdsourced labels from etherscan.
# Cannot scrape data from Etherscan: CloudFare protection.

